In [2]:
str_width = 40
import glob as glob
import xarray as xr
import cftime
import os

from functools import partial
def _preprocess(x, lat_bnds):
    return x.sel(lat=slice(*lat_bnds))

START_TIME = cftime.DatetimeNoLeap(3, 1, 4, 0, 0, 0, 0, has_year_zero=True)
END_TIME = cftime.DatetimeNoLeap(13, 1, 3, 0, 0, 0, 0, has_year_zero=True)
time_bounds = (START_TIME, END_TIME)
lat_bnds = (-40, 40)
partial_func = partial(_preprocess, lat_bnds=lat_bnds)

time_bounds = (START_TIME, END_TIME)

input_directory = "/glade/campaign/univ/uwas0114"
output_directory = "/glade/campaign/univ/uwas0152/post_processed_data"

variables_to_load = ['FLUT']

for experiment in ['0K']:

    print(f"Experiment: {experiment}")
    if experiment == '-4K':
        data_location = "SST_AQP3_Qobs_27_-4K_3h_10y"
    elif experiment == '0K':
        data_location = "SST_AQP3_Qobs_27_3h_20y"
    elif experiment == '4K':
        data_location = "SST_AQP3_Qobs_27_4K_3h_10y"

    data_files = sorted(glob.glob(f"{input_directory}/{data_location}/atm/hist/*h1*.nc"))

    variables_to_drop = [var for var in xr.open_dataset(data_files[0]).data_vars]
    for variable in variables_to_load:
        variables_to_drop.remove(variable)

    print(f"{f'Loading data...':<{str_width-1}}", end="")
    data = xr.open_mfdataset(
        data_files,
        drop_variables=variables_to_drop,
        preprocess=partial_func
    ).sel(time=slice(*time_bounds)).resample(time='1D').mean()
    print(rf"{'✔':>1}")

    print(f"{f'Saving data...':<{str_width-1}}")
    for index, (variable_name, variable_data) in enumerate(data.data_vars.items()):
        print(f"{f'({index+1}/{len(data.data_vars.items())}) {variable_name}...':<{str_width-1}}", end="")
        filename = f"{output_directory}/{experiment}/daily_model-level_data/SST_AQP3_Qobs_27_{experiment}_1D_{variable_name}.nc"
        if os.path.exists(filename):
            os.remove(filename)  # Delete the existing file
            variable_data.to_netcdf(filename)
            print(rf"{'✔ (overwritten)':>1}")
        else:
            variable_data.to_netcdf(filename)
            print(rf"{'✔':>1}")

    print(f"{'-'*str_width}")

print(f"{'='*str_width}")
print("Finished")

Experiment: 0K
Loading data...                        ✔
Saving data...                         
(1/1) FLUT...                          ✔ (overwritten)
----------------------------------------
Finished
